<a href="https://colab.research.google.com/github/jasonbuchanan145/Data-Modeling/blob/main/hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install huggingface_hub
!pip install py-solc-x
!pip install -U bitsandbytes
!pip install slither-analyzer
!solc-select install 0.8.28
!pip install trl
!solc-select use 0.8.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [2]:
from google.colab import files

# Create a working directory for our Solidity project
!mkdir -p solidity_project
%cd solidity_project

# Initialize npm project and install dependencies
!npm init -y
!npm install --save-dev hardhat @nomicfoundation/hardhat-toolbox @openzeppelin/contracts
!npm install @chainlink/contracts # Add other common dependencies as needed

with open('hardhat.config.js', 'w') as f:
    f.write('''module.exports = {
  solidity: "0.8.28",
  paths: {
    sources: "./contracts",
    tests: "./test",
    cache: "./cache",
    artifacts: "./artifacts"
  }
};''')

# Create contracts directory
!mkdir -p contracts

/content/solidity_project
Wrote to /content/solidity_project/package.json:

{
  "name": "solidity_project",
  "version": "1.0.0",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "description": ""
}



⠙⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋

In [3]:
import torch
from google.colab import drive
from transformers import AutoTokenizer, TFAutoModelForMaskedLM, GenerationConfig
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from datasets import load_dataset
import os
import re
import requests
from solcx import compile_standard, install_solc
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import matplotlib.pyplot as plt
import subprocess
import json
from torch.utils.data import Dataset
import torch.nn as nn
import tempfile
from torch.cuda.amp import autocast
from torch.amp import GradScaler
from bitsandbytes.optim import Adam8bit
# Mount Google Drive
drive.mount('/content/drive')
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# Set the path to save the best model
output_dir = "/content/drive/MyDrive/llama_generated_contracts/explanations"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
model_name = "Qwen/Qwen2.5-Coder-7B-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config
)
model.gradient_checkpointing_enable()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [5]:
# Create a new model instance on CPU
with torch.device("cpu"):
    reference_model = AutoModelForCausalLMWithValueHead.from_pretrained(
        model_name,
        device_map="cpu")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
from torch.utils.data import Dataset
import torch.nn as nn
from trl.core import LengthSampler
from datasets import Dataset as HFDataset
class SimpleRewardModel(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleRewardModel, self).__init__()
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, hidden_states):
        # For this simple example, we average the hidden states over the sequence length.
        mean_hidden = hidden_states.mean(dim=1)  # shape: [batch, hidden_size]
        reward = self.linear(mean_hidden)         # shape: [batch, 1]
        return reward

In [7]:
prompts = []
file_path = '/content/drive/MyDrive/prompts.txt'
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith('"') and line.endswith('"'):
            # Remove the surrounding quotes
            clean_line = line[1:-1]
            clean_line = "Using Solidity and having your code start with \"// Begin Smart Contract\" and ending with \"//End Smart Contract\" as comments "+clean_line
            prompts.append(clean_line)

In [8]:
# Function to compile a generated contract
def compile_contract(contract_source):
    # Write the contract to a file
    with open('contracts/MainContract.sol', 'w') as f:
        f.write(contract_source)

    # Run Hardhat compile
    !npx hardhat compile

    # Check if compilation was successful by looking for artifacts
    if os.path.exists('artifacts/contracts/MainContract.sol/'):
        print("Compilation successful!")
        return True
    else:
        print("Compilation failed!")
        return False

# Function to run Slither analysis
def run_slither():
    # Run Slither and capture output
    !slither contracts/MainContract.sol --json slither_output.json

    try:
        with open('slither_output.json', 'r') as f:
            results = json.load(f)
        return results
    except:
        print("Error running Slither or parsing results")
        return None

# Function to evaluate a contract (for reward model)
def evaluate_contract(contract_source):
    try:
        # Write contract to file
        with open('contracts/MainContract.sol', 'w') as f:
            f.write(contract_source)

        # Compile with Hardhat
        compile_result = !npx hardhat compile
        compilation_success = not any('Error' in line for line in compile_result)
        if not compilation_success:
            print("Compilation failed")
            print(compile_result)
            return -10
        return 10
        # Run Slither
        """
        !slither contracts/MainContract.sol --json slither_output.json
        try:
            with open('slither_output.json', 'r') as f:
                slither_results = json.load(f)

            print(slither_results)
            # Count issues by severity
            high_severity = sum(1 for d in slither_results.get('results', {}).get('detectors', [])
                               if d.get('impact') == 'High')
            medium_severity = sum(1 for d in slither_results.get('results', {}).get('detectors', [])
                                 if d.get('impact') == 'Medium')

            # Calculate reward (example scoring)
            base_reward = 10  # Base reward for successful compilation
            security_penalty = (high_severity * -3) + (medium_severity * -1)

            final_reward = base_reward + security_penalty
            return max(final_reward, -10)  # Cap minimum reward

        except Exception as e:
            print(f"Error processing Slither results: {e}")
            return 5
      """
    except Exception as e:
        print(f"Evaluation error: {e}")
        return -10

In [15]:
model.generation_config = GenerationConfig.from_model_config(model.config)

# Instantiate our simple reward model using the model's hidden size.
reward_model = SimpleRewardModel(hidden_size=model.config.hidden_size)

train_dataset = HFDataset.from_dict({"query": prompts})

# Define a prompt processor class (required by PPOTrainer)
class PromptProcessor:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def process_prompt(self, prompt):
        """Format prompt according to official Qwen pattern"""
        messages = [
            {"role": "system", "content": "You are an expert Solidity developer specialized in writing secure, efficient smart contracts. These contracts are intended to test your capabilities so ensure that you always provide complete, working code with no placeholders."},
            {"role": "user", "content": prompt}
        ]

        # Apply chat template exactly as in quickstart
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Create inputs as shown in quickstart
        model_inputs = self.tokenizer([text], return_tensors="pt").to("cuda:0")
        return model_inputs

    def generate_response(self, prompt_tensors, **generation_kwargs):
        # Define generation parameters
        gen_kwargs = {
            "max_new_tokens": 1000,
            "temperature": 0.7,
            "top_p": 0.95,
            "do_sample": True,
            "repetition_penalty": 1.1
        }
        gen_kwargs.update(generation_kwargs)

        # Generate response
        with torch.cuda.amp.autocast():  # Mixed precision for efficiency
            response = model.generate(**prompt_tensors, **gen_kwargs)

        return response

# Initialize processor with your tokenizer
prompt_processor = PromptProcessor(tokenizer)

formatted_prompts = [{"prompt": prompt} for prompt in prompts]
train_dataset = HFDataset.from_list(formatted_prompts)

ppo_config = PPOConfig(
    learning_rate=1e-5,
    batch_size=1,
    mini_batch_size=1,
    num_ppo_epochs=4,
    gamma=1.0,
    lam=0.95,
    cliprange=0.2,
    cliprange_value=0.2,
    vf_coef=0.1,
    seed=42,
    output_dir=output_dir
)
class CustomPPOTrainer(PPOTrainer):
    def __init__(self, *args, **kwargs):
        # Initialize without calling parent init as before
        from transformers import TrainingArguments
        self.args = kwargs.pop('args', None) or TrainingArguments("./output")
        self.policy_model = kwargs.pop('model')
        self.ref_model = kwargs.pop('ref_model')
        self.reward_function = kwargs.pop('reward_model')
        self.train_dataset = kwargs.pop('train_dataset')
        self.tokenizer = kwargs.pop('processing_class')
        self.policy_model.gradient_checkpointing_enable()
        self.setup_data_loader()
        # Initialize the GradScaler for mixed precision training
        self.scaler = GradScaler()
        self.setup_optimizers()  # Modified to use a memory-efficient optimizer

    def setup_data_loader(self):
        from torch.utils.data import DataLoader
        self.dataloader = DataLoader(
            self.train_dataset,
            batch_size=self.args.batch_size,
            shuffle=True
        )

    def setup_optimizers(self):
        # Instead of torch.optim.Adam, use the 8-bit Adam optimizer from bitsandbytes
        self.optimizer = Adam8bit(
            self.policy_model.parameters(),
            lr=self.args.learning_rate
        )

    def step(self, prompts, responses, rewards):
        # Cast input tensors to the appropriate dtype (float32) before computing loss
        if hasattr(responses, 'dtype') and responses.dtype == torch.float16:
            responses = responses.float()

        # Ensure prompts and rewards are properly typed
        if hasattr(prompts, 'dtype') and prompts.dtype == torch.float16:
            prompts = prompts.float()

        # Turn off autocast for the loss computation to avoid mixed precision issues
        with torch.cuda.amp.autocast(enabled=False):
            loss = self.compute_loss(prompts, responses, rewards)

        # Instead of using the scaler, just do a regular backward pass
        # This avoids the dtype issues with mixed precision
        loss.backward()

        for group in self.optimizer.param_groups:
            for p in group['params']:
                if p.grad is not None:
                   p.grad = p.grad.to(p.dtype)


        # Take optimizer step with the corrected gradients
        self.optimizer.step()
        self.optimizer.zero_grad(set_to_none=True)

        return {"loss": loss.item()}

    def compute_loss(self, prompt_ids, response_ids, rewards):
        # Limit sequence length
        max_length = min(256, response_ids.size(1))
        response_ids = response_ids[:, :max_length]

        # Create attention mask based on padding
        pad_id = self.tokenizer.pad_token_id
        attention_mask = (response_ids != pad_id).to(response_ids.device, dtype=torch.long)

        # Forward pass without mixed precision to ensure consistent dtypes
        outputs = self.policy_model(
            input_ids=response_ids,
            attention_mask=attention_mask,
            use_cache=False
        )
        logits = outputs[0] if isinstance(outputs, tuple) else getattr(outputs, "logits", outputs)

        # Get reference logits on CPU to save VRAM and move back to GPU
        with torch.no_grad():
            ref_outputs = self.ref_model(
                input_ids=response_ids.cpu(),
                attention_mask=attention_mask.cpu(),
                use_cache=False
            )
            ref_logits = ref_outputs[0] if isinstance(ref_outputs, tuple) else getattr(ref_outputs, "logits", ref_outputs)
            ref_logits = ref_logits.to(logits.device)

        # Compute log probabilities for policy and reference
        # Make sure both tensors are the same dtype
        policy_log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
        ref_log_probs = torch.nn.functional.log_softmax(ref_logits, dim=-1)

        # Ensure matching dtypes for KL div calculation
        if policy_log_probs.dtype != ref_log_probs.dtype:
            ref_log_probs = ref_log_probs.to(policy_log_probs.dtype)

        kl_div = torch.nn.functional.kl_div(
            policy_log_probs,
            ref_log_probs.exp(),  # ensure proper probability target
            reduction='none'
        ).sum(-1)

        # Compute per-token cross entropy loss
        gathered_log_probs = policy_log_probs.gather(dim=-1, index=response_ids.unsqueeze(-1)).squeeze(-1)
        nll_loss = -gathered_log_probs

        # Mask out padding tokens
        mask = (response_ids != pad_id).float()
        nll_loss = nll_loss * mask

        # Average loss per sequence (summing over tokens then averaging over the batch)
        token_loss = (nll_loss.sum(dim=1) / mask.sum(dim=1).clamp(min=1.0)).mean()

        # Convert rewards to tensor and weight the token loss
        rewards_tensor = torch.as_tensor(rewards, device=response_ids.device, dtype=torch.float32)
        weighted_loss = (token_loss * rewards_tensor).mean()

        # Final loss includes KL divergence penalty
        loss = weighted_loss + kl_div.mean() * ppo_config.kl_coef

        return loss

# Use the custom trainer if needed
ppo_trainer = CustomPPOTrainer(
    args=ppo_config,
    model=model,
    ref_model=reference_model,
    reward_model=evaluate_contract,
    train_dataset=train_dataset,
    processing_class=tokenizer
)

Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.


In [11]:
def generate_text(prompt, max_new_tokens=1000, temperature=0.2):
    """Generate text using the official Qwen pattern"""
    # Format messages properly
    messages = [
            {"role": "system", "content": "You are an expert Solidity developer specialized in writing secure, efficient smart contracts. "+
            "These contracts are intended to test your capabilities so ensure that you always provide complete, working code that compiles with no placeholders."},
            {"role": "user", "content": prompt}
        ]
    # Apply chat template as shown in quickstart
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Create inputs
    model_inputs = tokenizer([text], return_tensors="pt").to("cuda:0")

    # Generate with proper settings
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            repetition_penalty=1.1
        )

    # Extract only the new tokens - this is key!
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # Decode only the response portion
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [12]:
def extract_contract_code(response_text):
    """Extract the contract code from model response with improved pattern matching.
    Priority order:
    1. Find the last code block (```solidity ... ```)
    2. Look for Begin/End Smart Contract markers
    """
    import re

    # Case 1: Look for the LAST code block with optional 'solidity' language marker
    code_blocks = re.findall(r"```(?:solidity)?\s*(.*?)```", response_text, re.DOTALL)

    if code_blocks:
        # Get the last code block (highest priority)
        last_block = code_blocks[-1]
        if "contract" in last_block:
            start_marker = "// Begin Smart Contract"
            end_marker = "// End Smart Contract"
            return f"{start_marker}\n{last_block}\n{end_marker}"

    # Case 2: Look for markers outside or inside code blocks - use LAST occurrence
    start_marker = "// Begin Smart Contract"
    end_marker = "// End Smart Contract"

    # Find the LAST occurrence of the start marker
    start_idx = response_text.rfind(start_marker)

    # Find the LAST occurrence of the end marker
    end_idx = response_text.rfind(end_marker)

    if start_idx != -1 and end_idx != -1 and start_idx < end_idx:
        # Check if there's a closing bracket on the next line after the end marker
        end_marker_pos = end_idx + len(end_marker)
        next_closing_bracket = response_text[end_marker_pos:].find("}")

        if next_closing_bracket != -1 and response_text[end_marker_pos:end_marker_pos+next_closing_bracket].strip() == "":
            # Include the closing bracket if it's on the next line
            return response_text[start_idx:end_marker_pos + next_closing_bracket + 1]
        else:
            return response_text[start_idx:end_marker_pos]

    # Case 3: No markers and no code blocks
    print("No valid Solidity contract found in response")
    return None

In [13]:
def train(prompts, num_epochs=5):
    compilation_errors = [0 for _ in range(num_epochs)]
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        for i, prompt in enumerate(prompts):
            print(f"Processing prompt {i+1}/{len(prompts)}")

            # Step 1: Generate a response
            full_response = generate_text(prompt, max_new_tokens=1000, temperature=0.7)
            print(full_response)

            # Step 2: Extract the contract code
            contract_code = extract_contract_code(full_response)
            print("The final contract that we are going to try to compile is \n\n\n\n")
            print(contract_code)
            print("\nEnd Final \n\n\n\n")

            # Step 3: Calculate reward based on contract quality
            if contract_code is None:
                print("No contract code extracted, assigning negative reward")
                reward = -10
                if epoch < len(compilation_errors):
                    compilation_errors[epoch] += 1
            else:
                # Save contract to file for inspection (optional)
                contract_file = f"epoch_{epoch}_prompt_{i}_contract.sol"
                with open(contract_file, "w") as f:
                    f.write(contract_code)

                # Evaluate the contract
                reward = evaluate_contract(contract_code)
                print(f"Contract received reward: {reward}")
                if reward == -10 and epoch < len(compilation_errors):
                    compilation_errors[epoch] += 1

            # Step 4: Prepare inputs for PPO training
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
            response_ids = tokenizer(full_response, return_tensors="pt").input_ids.to("cuda:0")

            # Clear GPU memory
            torch.cuda.empty_cache()

            # Step 5: Use sliding window approach for PPO updates
            # The window size can be adjusted based on your GPU memory
            window_size = 256
            step_size = 128  # 50% overlap between windows

            # Process full response in overlapping chunks
            for start_idx in range(0, response_ids.size(1), step_size):
                end_idx = min(start_idx + window_size, response_ids.size(1))
                if end_idx - start_idx < 32:  # Skip very small chunks
                    continue

                window_response_ids = response_ids[:, start_idx:end_idx].to("cuda:0")

                # Apply the same reward to each window
                # This reinforces the entire generation rather than just the beginning
                stats = ppo_trainer.step(
                    inputs.input_ids,
                    window_response_ids,
                    [reward]
                )

                print(f"Window {start_idx}-{end_idx}: {stats}")

                # Optional: Clear GPU memory after each window
                torch.cuda.empty_cache()

    # Plot the results
    epochs = range(1, num_epochs + 1)
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, compilation_errors, marker='o', linestyle='-', linewidth=2)
    plt.title('Compilation Errors per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Number of Errors')
    plt.grid(True)
    plt.xticks(epochs)
    plt.savefig('compilation_errors.png')
    plt.show()

    return compilation_errors

In [ ]:
train(prompts)

Epoch 1/5
Processing prompt 1/110
```solidity
// Begin Smart Contract

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";
import "@openzeppelin/contracts/access/Ownable.sol";

contract TestCoin is ERC20, Ownable {
    constructor(uint256 initialSupply) ERC20("TestCoin", "TSC") {
        _mint(msg.sender, initialSupply);
    }

    function mint(address recipient, uint256 amount) external onlyOwner returns (bool) {
        _mint(recipient, amount);
        return true;
    }
}

// End Smart Contract
```

### Explanation:

1. **SPDX-License-Identifier**: This specifies the license under which the contract is released.
2. **pragma solidity ^0.8.0**: Specifies the version of Solidity to be used for compilation.
3. **import statements**: Imports necessary OpenZeppelin contracts for the ERC20 token functionality and ownership management.
4. **TestCoin Contract**:
   - Inherits from `ERC20` and `Ownable`.
   - The constructor initi

<ipython-input-15-b66cdcd6cdef>:108: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Compilation failed
['\x1b[1G\x1b[0K⠙\x1b[1G\x1b[0K\x1b\x1bTypeError:\x1b\x1b No arguments passed to the base constructor. Specify the arguments or mark "TestCoin" as abstract.', '  --> contracts/MainContract.sol:10:1:', '   |', '10 | contract TestCoin is ERC20, Ownable {', '   | ^ (Relevant source part starts here and spans across multiple lines).', 'Note: Base constructor parameters:', '  --> @openzeppelin/contracts/access/Ownable.sol:38:16:', '   |', '38 |     constructor(address initialOwner) {', '   |                ^^^^^^^^^^^^^^^^^^^^^^', '', '', '\x1b\x1bError\x1b\x1b \x1b\x1bHH600:\x1b\x1b Compilation failed', '', 'HardhatError: HH600: Compilation failed', '    at SimpleTaskDefinition.action (/content/solidity_project/node_modules/hardhat/src/builtin-tasks/compile.ts:499:15)', '    at Environment._runTaskDefinition (/content/solidity_project/node_modules/hardhat/src/internal/core/runtime-environment.ts:351:14)', '    at Environment.run (/content/solidity_project/node_modules/ha

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Window 0-256: {'loss': -189.68069458007812}


...

Window 128-384: {'loss': -190.77032470703125}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Window 256-512: {'loss': -194.86622619628906}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Window 384-604: {'loss': -166.8721160888672}
Window 512-604: {'loss': -147.73416137695312}
Processing prompt 10/110
